In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
from scipy.stats import norm

# binomial_tree_option_priceModel
from scipy.stats import norm
import numpy as np

# Binomial Options Pricing Model for American Call Option
def binomial_tree_option_price(S, K, T, r, sigma, N=1000):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    prices = np.zeros(N + 1)
    prices[0] = S * d ** N
    for i in range(1, N + 1):
        prices[i] = prices[i - 1] * u / d
    
    option_values = np.maximum(prices - K, 0)
    
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_values[j] = (p * option_values[j + 1] + (1 - p) * option_values[j]) * np.exp(-r * dt)
            option_values[j] = np.maximum(option_values[j], prices[j] - K)
    
    return option_values[0]

def calculate_greeks(S, K, T, r, sigma, N=100):
    base_price = binomial_tree_option_price(S, K, T, r, sigma, N)
    
    # Delta
    delta_S = 0.01 * S
    price_up = binomial_tree_option_price(S + delta_S, K, T, r, sigma, N)
    price_down = binomial_tree_option_price(S - delta_S, K, T, r, sigma, N)
    delta = (price_up - price_down) / (2 * delta_S)
    
    # Gamma
    price_up_up = binomial_tree_option_price(S + 2 * delta_S, K, T, r, sigma, N)
    gamma = (price_up_up - 2 * base_price + price_down) / (delta_S ** 2)
    
    # Theta
    delta_T = 1/365
    price_T_up = binomial_tree_option_price(S, K, T + delta_T, r, sigma, N)
    theta = (price_T_up - base_price) / delta_T
    
    # Vega
    delta_sigma = 0.01 * sigma
    price_sigma_up = binomial_tree_option_price(S, K, T, r, sigma + delta_sigma, N)
    vega = (price_sigma_up - base_price) / delta_sigma
    
    # Rho
    delta_r = 0.01
    price_r_up = binomial_tree_option_price(S, K, T, r + delta_r, sigma, N)
    rho = (price_r_up - base_price) / delta_r
    
    return delta, gamma, theta, vega, rho

# Retrieve historical data
ticker = "BTCC-B.TO"
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=365*4)).strftime('%Y-%m-%d')
end_time = (datetime.now(pytz.timezone('US/Pacific'))).strftime('%Y-%m-%d')
data = yf.download(ticker, start=start_time, end=end_time, interval="1d")[['Close']]
data['Daily_Return'] = data['Close'].pct_change()
daily_volatility = data['Daily_Return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

# Risk-free rate (annualized)
risk_free_rate = 0.05  # Example rate, adjust as needed

# Current stock price
current_stock_price = data['Close'].iloc[-1]

# Calculate the next 4 Fridays from today, spaced two weeks apart
today = datetime.now(pytz.timezone('US/Pacific')).date()
friday_dates = [today + timedelta(days=((4 - today.weekday()) % 7) + 30*i) for i in range(12)]

# Generate a range of strike prices from -50% to +50% of the current stock price by 5% increments
strike_price_range = np.arange(0, 1.0, 0.05)  # Ends at 0.55 to include 50%
strike_prices = [current_stock_price * (1 + x) for x in strike_price_range]

# Print options prices with improved format
last_expiry_date = None
for expiry_date in friday_dates:
    days_to_expiration = (expiry_date - today).days / 365  # Convert days to years
    if expiry_date != last_expiry_date:
        print(f"\nExpiration Date: {expiry_date.strftime('%Y-%m-%d')}")
        last_expiry_date = expiry_date
    for strike_price in strike_prices:
        call_option_price = binomial_tree_option_price(current_stock_price, strike_price, days_to_expiration, risk_free_rate, annualized_volatility)
        delta, gamma, theta, vega, rho = calculate_greeks(current_stock_price, strike_price, days_to_expiration, risk_free_rate, annualized_volatility)
        print(f"  Strike Price: {strike_price:.2f}, Call Option Price: {call_option_price:.2f}, Delta: {delta:.2f}, Gamma: {gamma:.2f}, Theta: {theta:.2f}, Vega: {vega:.2f}, Rho: {rho:.2f}")

[*********************100%%**********************]  1 of 1 completed
C:\Users\falty\AppData\Local\Temp\ipykernel_6848\615037854.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Daily_Return'] = data['Close'].pct_change()



Expiration Date: 2024-03-01
  Strike Price: 10.53, Call Option Price: 0.23, Delta: 0.51, Gamma: 6.69, Theta: 12.99, Vega: 0.38, Rho: 0.04
  Strike Price: 11.06, Call Option Price: 0.06, Delta: 0.19, Gamma: 3.10, Theta: 9.13, Vega: 0.26, Rho: 0.02
  Strike Price: 11.58, Call Option Price: 0.01, Delta: 0.04, Gamma: 0.83, Theta: 3.59, Vega: 0.09, Rho: 0.00
  Strike Price: 12.11, Call Option Price: 0.00, Delta: 0.01, Gamma: 0.13, Theta: 0.78, Vega: 0.02, Rho: 0.00
  Strike Price: 12.64, Call Option Price: 0.00, Delta: 0.00, Gamma: 0.01, Theta: 0.11, Vega: 0.00, Rho: 0.00
  Strike Price: 13.16, Call Option Price: 0.00, Delta: 0.00, Gamma: 0.00, Theta: 0.01, Vega: 0.00, Rho: 0.00
  Strike Price: 13.69, Call Option Price: 0.00, Delta: 0.00, Gamma: 0.00, Theta: 0.00, Vega: 0.00, Rho: 0.00
  Strike Price: 14.22, Call Option Price: 0.00, Delta: 0.00, Gamma: 0.00, Theta: 0.00, Vega: 0.00, Rho: 0.00
  Strike Price: 14.74, Call Option Price: 0.00, Delta: 0.00, Gamma: 0.00, Theta: 0.00, Vega: 0.00,